## Customer info requirements

At a high level the process is to:
1. Go to this website: https://www.cefconnect.com/
2. Enter a stock symbol
3. Go to the Pricing History page
4. Select the data for a specific month
5. Copy and paste the resulting data into a Google Sheet that I’ll provide.

Deliverables
Delivery of a Google Sheet with pricing data for all the 206 funds listed and for the 12 months requested.

### Importing Required Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import os
from bs4 import BeautifulSoup
import pandas as pd
from typing import List, Dict, Optional, Any, Union

### Data and link to search

In [3]:
url = "https://www.cefconnect.com/"
fund_name_list = ["ACP","FOF","EVT"]
funds_data = None

### Running the Scraper

In [ ]:
class Scrap:
    """Class to handle web scraping of fund data from CEF Connect website."""
    
    def __init__(self, url: str, fund_names: List[str]) -> None:
        """
        Initialize the scraper with target URL and fund names.
        
        Args:
            url: The website URL to scrape
            fund_names: List of fund symbols to search for
        """
        self.driver = webdriver.Chrome()
        self.months = list(range(1, 13))
        self.current_month = datetime.datetime.now().month
        self.fund_name_list = fund_names
        self.current_fund_name: Optional[str] = None
        self.funds_list: List[Dict[str, str]] = []
        self.url = url
        
        # Start the scraping process
        self.running()
    
    def running(self) -> None:
        """Main method to run the scraping process."""
        try:
            # Start browser and navigate to URL
            self.driver.get(self.url)
            
            # Wait for page to load
            self.driver.implicitly_wait(10)
            
            # Accept cookies banner if present
            cookies_close = self.driver.find_element(By.ID, "onetrust-accept-btn-handler")
            
            if cookies_close:
                time.sleep(5)
                cookies_close.click()

            # Check if fund list is empty
            if len(self.fund_name_list) <= 0:
                print("Fund List is empty")
                return

            # Process each fund in the list
            for fund in self.fund_name_list:
                time.sleep(5)
                self.current_fund_name = fund
                self.search_fund(fund)
                    
        except Exception as error:
            print(f"Error - Running: {error}")

    def search_fund(self, fund: str) -> None:
        """
        Search for a specific fund and extract its pricing data.
        
        Args:
            fund: Fund symbol to search for
        """
        try:
            # Find search input and enter fund name
            input_element = self.driver.find_element(By.ID, "quick-search-input-primary")
            input_element.clear()
            
            time.sleep(5)
            input_element.send_keys(fund)

            # Click search button
            time.sleep(5)
            self.driver.find_element(By.TAG_NAME, "button").click()

            # Navigate to pricing section
            time.sleep(5)
            self.driver.find_element(By.XPATH, "//a[@href='#pricing']").click()
            
            # Extract data for each month up to current month
            for month in self.months:
                if month <= 3:
                    time.sleep(2)
                    monthly_data = self.get_month(month)
                    if monthly_data:
                        self.funds_list.extend(monthly_data)

        except Exception as error:
            print(f"Error - Searching fund {fund}: {error}")

    def get_month(self, month: int, day: str = '01') -> Optional[List[Dict[str, str]]]:
        """
        Extract pricing data for a specific month.
        
        Args:
            month: Month number (1-12)
            day: Day of month (default: '01')
            
        Returns:
            List of dictionaries containing fund data for the specified month
        """
        try:
            # Find date input field
            date_input = self.driver.find_element(By.XPATH, '//*[@id="dp-phistory-date"]')
            
            time.sleep(4)
            
            # Format date and set it in the input field
            date_required = f'{month:02d}/{day}/2025'
            script = """
                arguments[0].removeAttribute('readonly');
                arguments[0].value = arguments[1]
            """
            self.driver.execute_script(script, date_input, date_required)
            time.sleep(5)

            # Click search button to load data
            go_btn = self.driver.find_element(By.XPATH, '//*[@id="btn-pricing-history"]')
            go_btn.click()

            # Extract table data
            time.sleep(5)
            table_element = self.driver.find_element(By.XPATH, '//*[@id="pricing-history"]/tbody')
            html_content = table_element.get_attribute('innerHTML')

            data_table = self.get_funds_data(html_content) #type: ignore
        
            # Display extracted data
            print({
                "Month": f'{month:02d}/{day}/2025',
                "Data": data_table
            })

            return data_table
            
        except Exception as error:
            print(f"Error - getting month {month}: {error}")
            return None

    def get_funds_data(self, html: str) -> List[Dict[str, str]]:
        """
        Parse HTML table content and extract fund data.
        
        Args:
            html: HTML content of the table
            
        Returns:
            List of dictionaries containing parsed fund data
        """
        try:
            soup = BeautifulSoup(html, "html.parser")
            time.sleep(2)
            rows = soup.find_all('tr')
            data = []

            if not rows:
                print("No data found: table info")
                return []

            # Process each row in the table
            for row in rows:
                cells = row.find_all("td")

                if not cells:
                    continue
                    
                row_data = self._create_data_dict(cells)
                data.append(row_data)

            return data

        except Exception as error:
            print(f"Error - getting data from table: {error}")
            return []

    def _create_data_dict(self, data_cells: List[Any]) -> Dict[str, str]:
        """
        Create a dictionary from table row data.
        
        Args:
            data_cells: List of table cell elements
            
        Returns:
            Dictionary containing fund data for one row
        """
        try:
            if len(data_cells) < 4:
                return {"Error": "Empty row in the table"}
            
            result = {
                "symbol": self.current_fund_name,
                "date": data_cells[0].get_text(strip=True),
                "share_price": data_cells[1].get_text(strip=True),
                "NAV": data_cells[2].get_text(strip=True),
                "Premiun|Discount": data_cells[3].get_text(strip=True)
            }
            return result
            
        except Exception as error:
            print(f"Error - Creating data dictionary: {error}")
            return {"Error": f"Failed to parse row data: {error}"}

# Execute the scraper
run = Scrap(url, fund_name_list)
funds_data = run.funds_list

### Total data obtained

In [15]:
len(funds_data)

605

### Getting Excel or CSV files

In [ ]:
def get_csv_xlxs(file,funds):
    try:
        
        # Make folder
        folder_path = "./files"
        os.makedirs(folder_path, exist_ok=True)
        
        # root
        root = f"{folder_path}/{file}"

        data = {
            "Symbol":[],
            "Date":[],
            "Share_Price":[],
            "Premiun|Discount":[],
        }

        for x in funds:
            data["Symbol"].append(x.get("symbol","None"))
            data["Date"].append(x.get("date","None"))
            data["Share_Price"].append(x.get("share_price","None"))
            data["Premiun|Discount"].append(x.get("Premiun|Discount","None"))

        df = pd.DataFrame(data)
        df.to_excel(f"{root}.xlsx",engine='openpyxl',index=False, sheet_name='funds')
        df.to_csv(f"{root}.csv",index=False, sep=";",encoding='utf-8-sig')

        print("Done")
        
    except Exception as error:
        print(f"Error CSV: {error}")

get_csv_xlxs("Funds",funds_data)

Done
